In [1]:
import json
import numpy as np
import pdb


In [2]:
# Recursive function to populate deeper reply levels

def NextLevel(replies, node_reply):
    if len(replies['replies'])>0:
        # Sort replies2
        for replies2 in replies['replies']:
            score_list.append(replies2['score'])
        ZipList = zip(score_list, replies['replies'])
        replies_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 
        # For each replies2 in sorted replies2
        for replies2 in replies_sorted:
            # Create node at replies-replies2
            node_replies2 = WNode(replies2['body'], parent=node_reply, weight=replies2['score'])
            # call NextLevel(replies2, node_replies2)
            NextLevel(replies2, node_replies2)



In [3]:
from anytree import NodeMixin, RenderTree, PreOrderIter

class WNode(NodeMixin):

     def __init__(self, foo, parent=None, weight=None):
         super(WNode, self).__init__()
         self.foo = foo
         self.parent = parent
         self.weight = weight if parent is not None else 0

     def _post_detach(self, parent):
         self.weight = None

episode_texts = []
episode_labels = []
episode_dones = []

with open('data_samples/reddit_female.json') as json_file:
    data = json.load(json_file)
    
    # Extracting Submissions
    for submission in data:
        episode_text = []
        episode_label = []
        episode_done = []
        
        title = submission['title']
        r = WNode(submission['body'])

        # Extracting Comments (Depth=0)
        score_list = []
        for comments in submission['comments']: # Ordering Comments
            score_list.append(comments['score'])
        ZipList = zip(score_list, submission['comments'])
        comments_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 
        # Scaling Scores to Max
        # if len(score_list)>0:
        #     score_max = max(score_list)
        # else:
            # score_max = 1
        score_max = 1
        
        for comments in comments_sorted:  # Creating sorted Comment-Nodes
            node_comment = WNode(comments['body'], parent=r, weight=comments['score']/score_max)

            # Extracting replies (Depth=1)
            score_list = []
            for replies in comments['replies']:
                score_list.append(replies['score'])
            ZipList = zip(score_list, comments['replies'])
            replies_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 

            for replies in replies_sorted:  # Creating sorted Reply-Nodes
                node_reply = WNode(replies['body'], parent=node_comment, weight=replies['score'])
                NextLevel(replies, node_reply)

        #  ------------------------------
        # Tree is built for this submission

        # Scoring every leaf path
        score_path_list = []
        leaves = list(PreOrderIter(r, filter_=lambda node: node.is_leaf))
        for leaf in leaves:
            leaf_path = leaf.path
            lenth_path = len(leaf_path)
            score_path = 0
            for node in leaf_path:
                score_path = score_path + node.weight
            score_path_list.append(score_path)
        score_path_list

        # Sorting every leaf path
        ZipList = zip(score_path_list, (leaf.path for leaf in leaves))
        leaf_path_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)]
        # leaf_path_sorted[1][2].foo[0:50] #path=1, node=2


        # labeling first N paths
        Npath_max = 5

        # leaf = leaf_path_sorted[0]
        upper = min(Npath_max, len(leaf_path_sorted))
        for leaf in leaf_path_sorted[0:upper]:
            for node in leaf:
                if len(node.children) == 0:
                    break
                elif len(node.children) == 1:
                    episode_texts.append(node.foo)
                    episode_labels.append(node.children[0].foo)
                    episode_dones.append(True)
                else:
                    episode_texts.append(node.foo)
                    episode_labels.append(node.children[0].foo)
                    episode_dones.append(False)

        episode_texts.append(episode_texts)
        episode_labels.append(episode_labels)
        episode_dones.append(episode_dones)

In [4]:
print('Number of episodes:')
len(episode_texts)

Number of episodes:


10822

In [5]:
# Visualising the sorted tree

for pre, _, node in RenderTree(r):
     print("%s%s (%s)" % (pre, node.foo[0:20], node.weight or 0))

I read this [NPR art (0)
├── > "house dresses," i (284.0)
│   ├── Oh, I didn't mean to (58)
│   │   └── I agree that the sty (39)
│   │       └── Yeah I used to feel  (14)
│   └── How do you find the  (3)
│       └── I got it in a clothi (1)
├── I read it slightly o (89.0)
│   ├── I actually went thro (15)
│   │   ├── [deleted] (34)
│   │   │   └── Yes, you're absolute (15)
│   │   │       └── [deleted] (23)
│   │   │           ├── As a very tall woman (-2)
│   │   │           │   └── [deleted] (3)
│   │   │           │       └── I'm in my 30s too an (2)
│   │   │           │           └── Super short girl her (2)
│   │   │           ├── Eff heels. Burn them (11)
│   │   │           └── I definitely think t (1)
│   │   └── Oh definitely not, b (1)
│   └── Link to your dress?  (1)
│       └── It was an h&m one. T (1)
├── I don’t equate House (57.0)
│   ├── Wish I could find a  (16)
│   │   ├── [Brooks Brothers ](h (15)
│   │   │   ├── If only the price wa (15)
│   │   │   │   └── Right!

In [35]:
# https://stackoverflow.com/questions/59917058/how-to-get-all-possible-branch-with-python-anytree

# Scoring every leaf path
score_path_list = []
leaves = list(PreOrderIter(r, filter_=lambda node: node.is_leaf))
for leaf in leaves:
    leaf_path = leaf.path
    lenth_path = len(leaf_path)
    score_path = 0
    for node in leaf_path:
        score_path = score_path + node.weight
    score_path_list.append(score_path)
score_path_list

# Sorting every leaf path
ZipList = zip(score_path_list, (leaf.path for leaf in leaves))
leaf_path_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)]
leaf_path_sorted[1][2].foo[0:50] #path=1, node=2


'How do you find the quality of the cupshe dress? I'

In [54]:
score_path_list_sorted = sorted(score_path_list, reverse=True)
# print(score_path_list_sorted)

# labeling first N paths
Npath_max = 10
episode_texts = []
episode_labels = []
episode_dones = []

# leaf = leaf_path_sorted[0]
upper = min(Npath_max, len(leaf_path_sorted))
for leaf in leaf_path_sorted[0:upper]:
    for node in leaf:
        if len(node.children) == 0:
            break
        elif len(node.children) == 1:
            episode_texts.append(node.foo)
            episode_labels.append(node.children[0].foo)
            episode_dones.append(True)
        else:
            episode_texts.append(node.foo)
            episode_labels.append(node.children[0].foo)
            episode_dones.append(False)

print('Example: ')    
print(episode_texts[7])
print('---')
print('Label: ')    
print(episode_labels[7])
print('---')
print('Done: ')    
print(episode_dones[7])

len(episode_texts)


Example: 
I read this [NPR article](https://www.npr.org/2020/08/18/902621260/the-house-dress-is-the-fashion-statement-of-2020) while working from home and wearing a sack dress, and I have a lot of thoughts.

From the article: 

>Billowing linen. Cozy cotton. Floating silk. The house dress is having a 2020 renaissance. Flowy tunics, chic kimonos and muumuus, and ankle-length T-shirts are floating into more and more shopping carts — a sartorial coping mechanism for the modern pandemic age.  
>  
>  
>  
>For decades, the house dress got a bad rep — a throwback to the times when women's sway was confined to housework. Its origin is said to trace to a Victorian gown that freed women of corsets but clad them instead in a baggy matronly smock named after "Mother Hubbard" from old nursery rhymes.

&#x200B;

I have definitely been wearing a lot of unstructured, loose dresses. But I have an issue with the idea of these dresses as "house dresses," i.e. clothes that you wouldn't want to be seen i

49

In [30]:
########################
######### SIDE #########
########################









                # reply2_list = replies['replies']
                # score_list = []
                # for reply2 in reply2_list:
                #     score_list.append(reply2['score'])
                # ZipList = zip(score_list, reply2_list)
                # replies_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 
                # for replies2 in replies_sorted:
                #     node_reply = WNode(replies2['body'], parent=node_reply, weight=replies2['score'])
                #     reply2_list = replies2['replies']






# def NextLevel(depth, replies, node_reply):
#     if len(replies['replies']) == 0:
#         node_reply = WNode(replies['body'], parent=node_reply, weight=replies['score'])
#     elif depth == 2:
#         score_list=[]
#         for replies2 in replies['replies']:
#             # breakpoint()
#             score_list.append(replies2['score'])
#         ZipList = zip(score_list, replies['replies'])
#         replies_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 
#         for replies2 in replies_sorted:
#             node_reply = WNode(replies2['body'], parent=node_reply, weight=replies2['score'])
#     else:
#         # breakpoint()
#         score_list=[]
#         for replies2 in replies['replies']:
#             score_list.append(replies2['score'])
#         ZipList = zip(score_list, replies['replies'])
#         replies_sorted = [x for _, x in sorted(ZipList, key=lambda pair: pair[0], reverse=True)] 
#         for replies2 in replies_sorted:
#             # breakpoint()
#             print(depth)
#             replies2 = replies2['replies']
#             NextLevel(depth+1, replies2, node_reply)

